In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
mydir = '/Users/supriyajadhav/Documents/MS422-Practical-Machine-Learning/Project422/'
processed_df = pd.read_csv(mydir + 'preprocessed_train_data.csv')
test_df = pd.read_csv(mydir + 'preprocessed_test_data.csv')

/var/folders/r1/dsylm7cn0vg3015hchpddz9m0000gn/T/ipykernel_82870/3545090819.py:2: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  processed_df = pd.read_csv(mydir + 'preprocessed_train_data.csv')


In [3]:
# Filter processed_df to only include rows where store_nbr is in (1, 2, 3, 4, 5)
processed_df = processed_df[processed_df['store_nbr'].isin([1])]

# Filter test_df to only include rows where store_nbr is in (1, 2, 3, 4, 5)]
test_df = test_df[test_df['store_nbr'].isin([1])]

result_file = 'sales_prediction_1.csv'

In [4]:
processed_df['date'] = pd.to_datetime(processed_df['date'])
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55704 entries, 0 to 3006530
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Unnamed: 0     55704 non-null  int64         
 1   id             55704 non-null  int64         
 2   date           55704 non-null  datetime64[ns]
 3   store_nbr      55704 non-null  int64         
 4   family         55704 non-null  object        
 5   sales          55704 non-null  float64       
 6   onpromotion    55704 non-null  int64         
 7   city           55704 non-null  object        
 8   state          55704 non-null  object        
 9   type           55704 non-null  object        
 10  cluster        55704 non-null  int64         
 11  holiday_type   4884 non-null   object        
 12  IS_HOLIDAY     55704 non-null  float64       
 13  oil_price      39732 non-null  float64       
 14  month          55704 non-null  int64         
 15  year           55

In [5]:
test_df['date'] = pd.to_datetime(test_df['date'])
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 528 entries, 0 to 26762
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Unnamed: 0     528 non-null    int64         
 1   id             528 non-null    int64         
 2   date           528 non-null    datetime64[ns]
 3   store_nbr      528 non-null    int64         
 4   family         528 non-null    object        
 5   onpromotion    528 non-null    int64         
 6   city           528 non-null    object        
 7   state          528 non-null    object        
 8   type           528 non-null    object        
 9   cluster        528 non-null    int64         
 10  IS_HOLIDAY     528 non-null    float64       
 11  day            528 non-null    int64         
 12  month          528 non-null    int64         
 13  year           528 non-null    int64         
 14  week_of_day    528 non-null    int64         
 15  week_of_month  528 no

In [6]:
unique_store_family_df = processed_df[['store_nbr', 'family']].drop_duplicates()

# Reset index if you want a clean DataFrame
unique_store_family_df = unique_store_family_df.reset_index(drop=True)

# Add a column for RMSLE values
unique_store_family_df['val_RMSLE'] = None

# Display the resulting DataFrame
print(len(unique_store_family_df))
print(unique_store_family_df.head())


33
   store_nbr      family val_RMSLE
0          1  AUTOMOTIVE      None
1          1   BABY CARE      None
2          1      BEAUTY      None
3          1   BEVERAGES      None
4          1       BOOKS      None


In [7]:
missing_data = processed_df.isnull().sum()
print(missing_data)

Unnamed: 0           0
id                   0
date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
city                 0
state                0
type                 0
cluster              0
holiday_type     50820
IS_HOLIDAY           0
oil_price        15972
month                0
year                 0
week_of_day          0
week_of_month        0
week_number          0
dtype: int64


In [8]:
processed_df = processed_df.drop(['holiday_type', 'oil_price'], axis = 1)

In [9]:
# Group train data
# Step 1: Group by 'store_nbr' and 'family'
grouped = processed_df.groupby(['store_nbr', 'family'])

# Step 2: Sort each group by 'date' in ascending order
sorted_df = grouped.apply(lambda x: x.sort_values('date')).reset_index(drop=True)

# Optional: If you want to create a dictionary of DataFrames, one for each group
grouped_dict = {f"processed_{store}_{family}": group.sort_values('date') 
                for (store, family), group in grouped}

In [10]:
# group test data
# Step 1: Group by 'store_nbr' and 'family'
grouped_test = test_df.groupby(['store_nbr', 'family'])

# Step 2: Sort each group by 'date' in ascending order
sorted_test_df = grouped_test.apply(lambda x: x.sort_values('date')).reset_index(drop=True)

# Optional: If you want to create a dictionary of DataFrames, one for each group
grouped_test_dict = {f"test_{store}_{family}": group.sort_values('date') 
                for (store, family), group in grouped_test}

/var/folders/r1/dsylm7cn0vg3015hchpddz9m0000gn/T/ipykernel_82870/3328157936.py:6: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  sorted_test_df = grouped_test.apply(lambda x: x.sort_values('date')).reset_index(drop=True)


In [11]:
import tensorflow as tf

# defien RMSLE metric
def rmsle(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float64)
    y_pred = tf.cast(y_pred, tf.float64)
    return tf.sqrt(tf.reduce_mean(tf.square(tf.math.log(y_pred + 1) - tf.math.log(y_true + 1))))


In [12]:
predictions_list=[]

In [13]:
# Loop through each unique combination
for _, row in unique_store_family_df.iterrows():
    store = row['store_nbr']
    family = row['family']

    # Get the corresponding train subset DataFrame
    subset_df = grouped_dict[f'processed_{store}_{family}']

    # Extract date components
    subset_df['day'] = subset_df['date'].dt.day

    # Create cyclical features
    subset_df['month_sin'] = np.sin(2 * np.pi * subset_df['month'] / 12)
    subset_df['month_cos'] = np.cos(2 * np.pi * subset_df['month'] / 12)
    subset_df['weekday_sin'] = np.sin(2 * np.pi * subset_df['week_of_day'] / 7)
    subset_df['weekday_cos'] = np.cos(2 * np.pi * subset_df['week_of_day'] / 7)

    # Drop rows with NaN values if needed
    subset_df = subset_df.dropna()

    # drop columns having many nulls
    subset_df = subset_df.drop(['Unnamed: 0', 'date', 'id', 'city', 'state', 'store_nbr', 'family', 'type' ], axis = 1)

    # Convert data to float64
    subset_df = subset_df.astype('float64')

    # Selecting features and target
    features = ['onpromotion', 'cluster', 'IS_HOLIDAY', 'month', 'year',
       'week_of_day', 'week_of_month', 'week_number', 'day', 'month_sin',
       'month_cos', 'weekday_sin', 'weekday_cos']
    
    target = 'sales'
    
    # Get the records for the training set except for last 15
    X = subset_df[features]
    y = subset_df[target]

    # Splitting data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

    # Feature scaling
    feature_scaler = MinMaxScaler(feature_range=(0, 1))
    target_scaler = MinMaxScaler(feature_range=(0, 1))

    X_train_scaled = feature_scaler.fit_transform(X_train)
    X_test_scaled = feature_scaler.fit_transform(X_test)
    y_train_scaled = target_scaler.fit_transform(y_train.values.reshape(-1,1))
    y_test_scaled = target_scaler.fit_transform(y_test.values.reshape(-1,1))

    # LSTM Model
    print(f'\n Training for store {store} and family {family} \n')  

    # Reshape data to fit LSTM input shape (num_samples, timesteps, num_features)
    timesteps = 1  # You need to define this based on your data and model requirements
    X_train_reshaped = X_train_scaled.reshape((X_train_scaled.shape[0], timesteps, X_train_scaled.shape[1]))
    X_test_reshaped = X_test_scaled.reshape((X_test_scaled.shape[0], timesteps, X_test_scaled.shape[1]))

    # Building the LSTM model
    lstm_model = Sequential()
    lstm_model.add(LSTM(units=60, return_sequences=True, input_shape=(timesteps, X_train_reshaped.shape[2])))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(LSTM(units=60, return_sequences=False))
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(units=1))

    lstm_model.compile(optimizer='adam', loss=rmsle)
    #lstm_model.summary()

    # Define early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Training the LSTM model
    lstm_model.fit(X_train_reshaped, y_train_scaled, epochs=50, batch_size=32, 
               validation_data=(X_test_reshaped, y_test_scaled),
               callbacks=[early_stopping])

    ## Predictions#####################################
    lstm_predictions = lstm_model.predict(X_test_reshaped)

    # Ensure both predictions and actual values are of type float64
    y_test_scaled = y_test_scaled.reshape(-1, 1).astype('float64')
    lstm_predictions_scaled = lstm_predictions.astype('float64')

    # Calculate RMSLE
    lstm_rmsle_val = rmsle(y_test_scaled, lstm_predictions_scaled)

    unique_store_family_df.loc[
        (unique_store_family_df['store_nbr'] == store) &
        (unique_store_family_df['family'] == family),
        'val_RMSLE'
        ] = lstm_rmsle_val.numpy()

    print(f"LSTM RMSLE for store: {store} and family: {family} is : {lstm_rmsle_val.numpy()}")

    # # last 15 days from train data are traeted as test
    # test_reshaped = test_df.values.reshape((test_df.shape[0], timesteps, test_df.shape[1]))
    # test_df_target_reshaped = test_df_target.values.reshape(-1, 1)
    
    # # Make predictions using the LSTM model
    # test_df_target_reshaped = test_df_target_reshaped.astype('float64')
    # lstm_predictions_test = lstm_model.predict(test_reshaped)

    # # Ensure both predictions and actual values are of type float64
    # test_df_target = target_scaler.inverse_transform(test_df_target_reshaped).astype('float64')
    # lstm_predictions_test = target_scaler.inverse_transform(lstm_predictions_test).astype('float64')

    # # Calculate RMSLE
    # lstm_rmsle_test = rmsle(test_df_target, lstm_predictions_test)

    # unique_store_family_df.loc[
    #     (unique_store_family_df['store_nbr'] == store) &
    #     (unique_store_family_df['family'] == family),
    #     'test_RMSLE'
    #     ] = lstm_rmsle_test.numpy()

    # print(f"LSTM RMSLE on test data: {lstm_rmsle_test.numpy()}")


    # perform data pre processing on kaggel test.csv#######################################

    # Get the corresponding test subset DataFrame
    test_subset_df = grouped_test_dict[f'test_{store}_{family}']

    # Create cyclical features
    test_subset_df['month_sin'] = np.sin(2 * np.pi * test_subset_df['month'] / 12)
    test_subset_df['month_cos'] = np.cos(2 * np.pi * test_subset_df['month'] / 12)
    test_subset_df['weekday_sin'] = np.sin(2 * np.pi * test_subset_df['week_of_day'] / 7)
    test_subset_df['weekday_cos'] = np.cos(2 * np.pi * test_subset_df['week_of_day'] / 7)

    # Drop rows with NaN values if needed
    test_subset_df = test_subset_df.dropna()

    test_subset_df_cp = test_subset_df.copy()

    # drop columns having many nulls
    test_subset_df_cp = test_subset_df_cp.drop(['Unnamed: 0', 'date', 'id', 'city', 'state', 'store_nbr', 'family', 'type' ], axis = 1)

    # Convert data to float64
    test_subset_df_cp = test_subset_df_cp.astype('float64')

    # feature scaling
    test_subset_df_cp_scaled = feature_scaler.fit_transform(test_subset_df_cp)


    # resahaping for LSTM
    test_reshaped = test_subset_df_cp_scaled.reshape((test_subset_df_cp_scaled.shape[0], 
                                                      timesteps, 
                                                      test_subset_df_cp_scaled.shape[1]))

    # Make predictions using the LSTM model
    lstm_predictions_test_normalised = lstm_model.predict(test_reshaped)

    # inverse transform
    lstm_predictions_test_normalised_reshaped = lstm_predictions_test_normalised.astype('float64')
    lstm_predictions_test_actual = target_scaler.inverse_transform(lstm_predictions_test_normalised_reshaped).astype('float64')

    predictions_df = pd.DataFrame({
        'id': test_subset_df['id'],
        'store_nbr': store,
        'family': family,
        'date': test_subset_df['date'], 
        'predictions': lstm_predictions_test_actual.flatten(),
        'val_rmsle': unique_store_family_df['val_RMSLE']   # Ensure predictions is 1D
    })

    # Round predictions to 4 decimal places and replace negative values with 0
    predictions_df['predictions'] = predictions_df['predictions'].round(4)
    predictions_df['predictions'] = predictions_df['predictions'].apply(lambda x: max(0, x))

    predictions_list.append(predictions_df)



 Training for store 1 and family AUTOMOTIVE 

Epoch 1/50


/Users/supriyajadhav/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1449 - val_loss: 0.1207
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1156 - val_loss: 0.1161
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1107 - val_loss: 0.1163
Epoch 4/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1128 - val_loss: 0.1166
Epoch 5/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1159 - val_loss: 0.1179
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
LSTM RMSLE for store: 1 and family: AUTOMOTIVE is : 0.1169706589710325
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step

 Training for store 1 and family BABY CARE 

Epoch 1/50


/Users/supriyajadhav/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.0099 - val_loss: 0.0015
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0047 - val_loss: 8.5481e-04
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0034 - val_loss: 7.3257e-04
Epoch 4/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0024 - val_loss: 2.8298e-04
Epoch 5/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0019 - val_loss: 3.6675e-04
Epoch 6/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0015 - val_loss: 7.4479e-04
Epoch 7/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0013 - val_loss: 2.5256e-04
Epoch 8/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 9.4349e-04 - val_loss: 3.3598e-04
Epoch 9/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 8.0770e-04 - val_loss: 8.0559e-05
Epoch 10/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 6.5231e-04 - val_loss: 1.9404e-04
Epoch 11/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 5.5717e-04 - val_loss: 5.5273e-04
Epoch 12/50
43/43 ━━━━━━━━━━━

...

43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2416 - val_loss: 0.1954
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1892 - val_loss: 0.1702
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1588 - val_loss: 0.1583
Epoch 4/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1416 - val_loss: 0.1370
Epoch 5/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1454 - val_loss: 0.1582
Epoch 6/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1493 - val_loss: 0.1447
Epoch 7/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1402 - val_loss: 0.1434
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
LSTM RMSLE for store: 1 and family: PRODUCE is : 0.1530711990395311
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step

 Training for store 1 and family SCHOOL AND OFFICE SUPPLIES 

Epoch 1/50


/Users/supriyajadhav/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0813 - val_loss: 0.1400
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0673 - val_loss: 0.1394
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0638 - val_loss: 0.1399
Epoch 4/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0755 - val_loss: 0.1403
Epoch 5/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0837 - val_loss: 0.1415
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
LSTM RMSLE for store: 1 and family: SCHOOL AND OFFICE SUPPLIES is : 0.16026602093974573
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step

 Training for store 1 and family SEAFOOD 

Epoch 1/50


/Users/supriyajadhav/anaconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


43/43 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.2056 - val_loss: 0.1045
Epoch 2/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.1006 - val_loss: 0.0874
Epoch 3/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0918 - val_loss: 0.0787
Epoch 4/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0833 - val_loss: 0.0767
Epoch 5/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0852 - val_loss: 0.0764
Epoch 6/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0782 - val_loss: 0.0759
Epoch 7/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0808 - val_loss: 0.0752
Epoch 8/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0783 - val_loss: 0.0754
Epoch 9/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0803 - val_loss: 0.0752
Epoch 10/50
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0788 - val_loss: 0.0754
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
LSTM RMSLE for store: 1 and family: SEAFOOD is : 0.07630646503949363
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


In [14]:
final_predictions_df = pd.concat(predictions_list, ignore_index=True)
final_predictions_df.to_csv(mydir + result_file)

In [18]:
import os
import glob

# Find all files starting with 'sales_prediction_' and read them into DataFrames
file_list = glob.glob(os.path.join(mydir, 'sales_prediction_*.csv'))
final_df = pd.concat((pd.read_csv(file, usecols=['id', 'predictions']) for file in file_list), ignore_index=True)

# Sort by 'id' and rename 'predictions' to 'sales'
final_df = final_df.sort_values(by='id').reset_index(drop=True)
final_df.rename(columns={'predictions': 'sales'}, inplace=True)

# Save the final DataFrame to a CSV file
final_df.to_csv(os.path.join(mydir, 'final_sales_predictions.csv'), index=False)

In [19]:
# Display the first few rows
print(final_df.head())
print(len(final_df))

        id     sales
0  3000888    1.7795
1  3000889    0.0000
2  3000890    2.7419
3  3000891  831.4846
4  3000892    0.0085
28512


In [20]:
mean_rmsle = unique_store_family_df['val_RMSLE'].sum() / len(unique_store_family_df)

print(f'The mean RMSLE for the LSTM Model over 1782 series is {mean_rmsle}')
print('Training & Inference duration: 3114 seconds')

The mean RMSLE for the LSTM Model over 1782 series is 0.14207708450767714
Training & Inference duration: 3114 seconds
